In [ ]:
from database_handler import *

In [ ]:
session = create_connection()

In [2]:
from transformation_handler import *

In [3]:
mydata = clean_all_data(1000000)

c:\Users\olasm\anaconda3\envs\etl_env\lib\site-packages\pandas\core\arrays\timedeltas.py:908: RuntimeWarning: invalid value encountered in cast
  base = data.astype(np.int64)
c:\Users\olasm\anaconda3\envs\etl_env\lib\site-packages\pandas\core\arrays\timedeltas.py:912: RuntimeWarning: invalid value encountered in cast
  data = (base * m + (frac * m).astype(np.int64)).view("timedelta64[ns]")


In [5]:
import datetime
sonoma = f"https://data.sonomacounty.ca.gov/resource/924a-vesw.json?$select=id,type,breed,color,sex,date_of_birth,intake_date,outcome_date,intake_type,outcome_type"

In [19]:
print(datetime.datetime.now())

2023-10-12 19:07:37.802788


In [7]:
response = requests.get(sonoma, params={'$limit': 1000000,'$where': "intake_date >= '2023-10-08T19:07:37.802788' or outcome_date >= '2023-10-08T19:07:37.802788'",'$order': 'intake_date ASC'})
df = pd.DataFrame(response.json())
dfs

,id,type,breed,color,sex,date_of_birth,intake_date,outcome_date,intake_type,outcome_type
0,A419496,DOG,WELSH CORGI CAR/MIX,WHITE/BROWN,Neutered,2020-09-19T00:00:00.000,2023-08-05T00:00:00.000,2023-10-11T00:00:00.000,STRAY,ADOPTION
1,A419911,DOG,MALTESE/MIX,WHITE,Male,2015-09-21T00:00:00.000,2023-08-25T00:00:00.000,2023-10-11T00:00:00.000,STRAY,EUTHANIZE
2,A420030,DOG,BELG MALINOIS/MIX,BLACK/TAN,Spayed,2023-06-03T00:00:00.000,2023-08-29T00:00:00.000,2023-10-10T00:00:00.000,STRAY,ADOPTION
3,A420029,DOG,BELG MALINOIS/MIX,BLACK/TAN,Neutered,2023-06-03T00:00:00.000,2023-08-29T00:00:00.000,2023-10-10T00:00:00.000,STRAY,ADOPTION
4,A420078,DOG,CHIHUAHUA SH,TAN/WHITE,Spayed,2020-10-10T00:00:00.000,2023-08-31T00:00:00.000,2023-10-11T00:00:00.000,STRAY,ADOPTION
5,A420076,CAT,DOMESTIC MH,BLACK/WHITE,Neutered,2019-08-31T00:00:00.000,2023-08-31T00:00:00.000,2023-10-11T00:00:00.000,STRAY,ADOPTION
6,A420165,CAT,DOMESTIC SH,BRN TABBY,Male,2023-07-05T00:00:00.000,2023-09-05T00:00:00.000,2023-10-10T00:00:00.000,STRAY,TRANSFER
7,A420194,DOG,PATTERDALE TERR/MIX,BLACK,Neutered,2023-04-04T00:00:00.000,2023-09-06T00:00:00.000,2023-10-10T00:00:00.000,STRAY,ADOPTION
8,A420226,CAT,DOMESTIC SH,ORG TABBY,Spayed,2021-09-07T00:00:00.000,2023-09-07T00:00:00.000,2023-10-11T00:00:00.000,STRAY,ADOPTION
9,A420681,DOG,GOLDEN RETR,GOLD,Female,2019-10-05T00:00:00.000,2023-09-26T00:00:00.000,2023-10-11T00:00:00.000,STRAY,TRANSFER
